# Import

In [1]:
import nii_io as nio
import nii_utils as utils
import nii_settings as nst
import nii_MDV as mdv
import numpy as np
from sklearn.mixture import GMM
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure, show, axes, sci
from IPython.display import display, Image #Config the matlotlib backend as plotting inline in IPython
%matplotlib inline

# Settings

In [2]:
proj_dir = "."
nii_dir = "./brainNii"
nii_segm_dir = "./brainNii/segmentation"
nii_settings_name = "settings.json"
nii_data_name = "patterns.json"
nii_histogram_name = "histogram.json"
nii_output_dir = "./brainNii/output"

extensions = ['.nii']
patch_side = 3 # considering square pathces

settings = utils.loadDict(proj_dir,nii_settings_name)
if settings == None:
    settings = nst.createSettings(nii_dir,extensions)
utils.saveDict(proj_dir,nii_settings_name,settings)
    
data = utils.loadDict(proj_dir,nii_data_name) # load data 
data = {} if data == None else data

print 'sampled data size'
print '3x3x3 patterns 2 bits per pixel = 54 bits per pattern'
print 'numeber of patter sampled: ' + `len(data.keys())`
print 'total size of pattern sampled: ' + `((len(data.keys())* 54) / 8) /1024` + ' Kbytes'

print settings

sampled data size
3x3x3 patterns 2 bits per pixel = 54 bits per pattern
numeber of patter sampled: 0
total size of pattern sampled: 0 Kbytes
{'todo_dirlist': ['AD-7.nii'], 'done_dirlist': ['AD-1.nii', 'CTRL-10.nii', 'CONV-30.nii', 'NOCONV-50.nii', 'NOCONV-10.nii', 'CTRL-5.nii'], 'thress': {'AD-7.nii': [0.1253376906090581, 0.32831875008302286, 0.5264969822905372, 1.0485159228165726]}}


# Starting sampling

In [4]:
samplers = {}
for img_name in settings['todo_dirlist']:
    samplers[img_name]=  mdv.sampler(3,settings['thress'][img_name]) 
print samplers

sample = {}
for img_name in settings['todo_dirlist']:
    image_data_and_metadata = (nio.basicLoadNii(utils.joinpath(nii_dir,img_name)))
    %time curr_sample = samplers[img_name].sample3D(image_data_and_metadata[2],verbose=True)
    sample = mdv.mergedicts(curr_sample,sample)
    
print len(sample)

{'AD-7.nii': <nii_MDV.nii_MDV.sampler instance at 0x7f523887d368>}
start
progress:  100 %
ended
CPU times: user 1min 19s, sys: 384 ms, total: 1min 19s
Wall time: 1min 18s
1237648
